# Optimizing Model Parameters

## prerequesite code 

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

100%|███████████████████████████████████████████████████████████████████████████| 26421880/26421880 [00:28<00:00, 922722.05it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|█████████████████████████████████████████████████████████████████████████████████| 29515/29515 [00:00<00:00, 279922.49it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|████████████████████████████████████████████████████████████████████████████| 4422102/4422102 [00:00<00:00, 5134309.05it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████████████████████████████████████████████████████████████████████████████| 5148/5148 [00:00<00:00, 3920878.34it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



## Hyper parameters

In [2]:
'''
Number of Epochs - the number of times to iterate over the dataset

Batch Size - the number of data samples propagated through the network before the parameters are updated

Learning Rate - how much to update models parameters at each batch/epoch. Smaller values 
yield slow learning speed, while large values may result in unpredictable behavior during training.
'''

learning_rate = 1e-3
batch_size = 64
epochs = 5

## Optimization loop

In [3]:
'''
Once we set our hyperparameters, we can then train and optimize our model with an optimization loop. 
Each iteration of the optimization loop is called an epoch.

Each epoch consists of two main parts:
The Train Loop - iterate over the training dataset and try to converge to optimal parameters.

The Validation/Test Loop - iterate over the test dataset to check if model performance is improving.
'''



'\nOnce we set our hyperparameters, we can then train and optimize our model with an optimization loop. \nEach iteration of the optimization loop is called an epoch.\n\nEach epoch consists of two main parts:\nThe Train Loop - iterate over the training dataset and try to converge to optimal parameters.\n\nThe Validation/Test Loop - iterate over the test dataset to check if model performance is improving.\n'

## Loss Function

In [4]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

## Optimizer

In [5]:
'''
We initialize the optimizer by registering the model’s parameters that need to be trained, and passing in the 
learning rate hyperparameter.

'''

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

## Full implementation

In [7]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.322438  [   64/60000]
loss: 2.306354  [ 6464/60000]
loss: 2.281633  [12864/60000]
loss: 2.265416  [19264/60000]
loss: 2.248285  [25664/60000]
loss: 2.225813  [32064/60000]
loss: 2.220532  [38464/60000]
loss: 2.196877  [44864/60000]
loss: 2.192961  [51264/60000]
loss: 2.146403  [57664/60000]
Test Error: 
 Accuracy: 41.2%, Avg loss: 2.153532 

Epoch 2
-------------------------------
loss: 2.173049  [   64/60000]
loss: 2.160128  [ 6464/60000]
loss: 2.097641  [12864/60000]
loss: 2.107674  [19264/60000]
loss: 2.059911  [25664/60000]
loss: 2.001401  [32064/60000]
loss: 2.022078  [38464/60000]
loss: 1.947371  [44864/60000]
loss: 1.953545  [51264/60000]
loss: 1.871868  [57664/60000]
Test Error: 
 Accuracy: 57.7%, Avg loss: 1.877313 

Epoch 3
-------------------------------
loss: 1.915200  [   64/60000]
loss: 1.880644  [ 6464/60000]
loss: 1.761675  [12864/60000]
loss: 1.802409  [19264/60000]
loss: 1.697270  [25664/60000]
loss: 1.646358  [32064/600